In [1]:
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.options.display.max_columns = 110


In [45]:
byCategory=pd.read_csv('0614byCategory.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [46]:
##將檔期時間調整為至少一個月(檔期開始時間最晚為2019-03-30)
byCategory=byCategory[byCategory.business_hour_order_time_s<'2019-03-30']

In [47]:
##找出檔期長度超過3年,檔期長度即設為1100天
byCategory.loc[byCategory.campagin_period>1100,'campagin_period']=1100

In [48]:
byCategory=byCategory[['business_hour_guid','product_launch_month', 'product_launch_hour',
       'product_launch_weekday','code', 'parent_code', 'is_Presales', 'is_Longtern',  
        'enable_isp','is_Discount','installment_3months', 'installment_6months', 'installment_12months',
       'deny_installment', 
       'item_orig_price', 'item_price','item_default_daily_amount',
       'quantity_multiplier','shippingdate', 'freight_amount',
        'campagin_period', 'order_deliver_days','unit_price',
       'category_unit_price', 'compare_cate_unit_price',
       'guid_subtotal', 'quantity_sold_guid', 'subtotal_allguid',
       'quantity_sold_allguid', 'avg_unit_price', 'price_gap',
        'hottimes']]


In [49]:
byCategory['unit_price']=byCategory['item_price']/byCategory['quantity_multiplier']


In [50]:
byCategory.columns

Index(['business_hour_guid', 'product_launch_month', 'product_launch_hour',
       'product_launch_weekday', 'code', 'parent_code', 'is_Presales',
       'is_Longtern', 'enable_isp', 'is_Discount', 'installment_3months',
       'installment_6months', 'installment_12months', 'deny_installment',
       'item_orig_price', 'item_price', 'item_default_daily_amount',
       'quantity_multiplier', 'shippingdate', 'freight_amount',
       'campagin_period', 'order_deliver_days', 'unit_price',
       'category_unit_price', 'compare_cate_unit_price', 'guid_subtotal',
       'quantity_sold_guid', 'subtotal_allguid', 'quantity_sold_allguid',
       'avg_unit_price', 'price_gap', 'hottimes'],
      dtype='object')

In [51]:
df=byCategory
df.shape

(224074, 32)

In [55]:
#發現 code,category_unit_price,compare_cate_unit_price 有nan值 因為分類關係
df.code=df.code.fillna(0)
df.shippingdate=df.shippingdate.fillna(0)
df.isna().sum()

business_hour_guid           0
product_launch_month         0
product_launch_hour          0
product_launch_weekday       0
code                         0
parent_code                  0
is_Presales                  0
is_Longtern                  0
enable_isp                   0
is_Discount                  0
installment_3months          0
installment_6months          0
installment_12months         0
deny_installment             0
item_orig_price              0
item_price                   0
item_default_daily_amount    0
quantity_multiplier          0
shippingdate                 0
freight_amount               0
campagin_period              0
order_deliver_days           0
unit_price                   0
category_unit_price          0
compare_cate_unit_price      0
guid_subtotal                0
quantity_sold_guid           0
subtotal_allguid             0
quantity_sold_allguid        0
avg_unit_price               0
price_gap                    0
hottimes                     0
dtype: i

In [56]:
df['category_unit_price']=df['category_unit_price'].fillna(df['unit_price'])

In [57]:
df['compare_cate_unit_price']=df['compare_cate_unit_price'].fillna(1)

## 將檔案順序打散

In [58]:
df=df.sample(frac=1, replace=False, random_state=1)

In [59]:
df=df.reset_index(drop=True)

將資料分為輸入變數與目標變數

## 特徵欄位進行標準化

In [60]:
df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,code,parent_code,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,536.0,536.0,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0
1,F0AB8354-EB2A-44F6-87D7-F71DAFFF1D28,6,7,3,166.0,166.0,0,0,0,1,0,0,0,0,890.0,239.0,20,1.0,0.0,0.0,13,0,239.0,359.0,0.666,4302.0,18.0,0.0,0.0,239.0,651.0,1
2,D7EE78D5-DE57-4486-881C-1DF8D8B220BC,9,7,2,1604.0,166.0,0,0,0,1,1,0,0,0,799.0,429.0,20,1.0,7.0,0.0,21,0,429.0,319.0,1.345,2574.0,6.0,0.0,0.0,429.0,370.0,0


In [61]:
target=pd.read_csv('target.csv')
code=pd.read_csv('code_time.csv')
target.head(3)

,business_hour_guid,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days
0,00000793-A001-4C8C-9196-3E6971181246,399.0,399.0,1.0,399.0,1.0,399.0,1.0,399.0,1.0,399.0,1.0
1,00001911-C3DF-4143-BC25-8AA5FA1A9AF4,1150.0,1150.0,1.0,1150.0,1.0,1150.0,1.0,1150.0,1.0,1150.0,1.0
2,00001928-AE75-4AA9-8A47-4BDE96D08C2D,23040.0,0.0,0.0,11520.0,0.5,23040.0,1.0,23040.0,1.0,23040.0,1.0


In [62]:
df=pd.merge(df,target,how='left')
df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,code,parent_code,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,536.0,536.0,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.200,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0
1,F0AB8354-EB2A-44F6-87D7-F71DAFFF1D28,6,7,3,166.0,166.0,0,0,0,1,0,0,0,0,890.0,239.0,20,1.0,0.0,0.0,13,0,239.0,359.0,0.666,4302.0,18.0,0.0,0.0,239.0,651.0,1,4302.0,2390.0,0.556,4302.0,1.000,4302.0,1.0,4302.0,1.0,4302.0,1.0
2,D7EE78D5-DE57-4486-881C-1DF8D8B220BC,9,7,2,1604.0,166.0,0,0,0,1,1,0,0,0,799.0,429.0,20,1.0,7.0,0.0,21,0,429.0,319.0,1.345,2574.0,6.0,0.0,0.0,429.0,370.0,0,2574.0,1287.0,0.500,1287.0,0.500,2574.0,1.0,2574.0,1.0,2574.0,1.0


In [63]:
df=pd.merge(df,code)
df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,code,parent_code,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days,code_total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days,code_max_7days,code_min_7days,code_median_7days,code_mean_7days,code_max_14days,code_min_14days,code_median_14days,code_mean_14days,code_max_30days,code_min_30days,code_median_30days,code_mean_30days,code_max_60days,code_min_60days,code_median_60days,code_mean_60days,code_max_180days,code_min_180days,code_median_180days,code_mean_180days
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,536.0,536.0,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.2,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0
1,9584B6E4-264C-4FFC-8EF6-5223301E692A,9,7,4,536.0,536.0,0,1,0,1,1,0,0,0,1380.0,799.0,20,1.0,0.0,0.0,81,0,799.0,299.0,2.672,3995.0,5.0,0.0,0.0,799.0,581.0,0,3995.0,799.0,0.2,799.0,0.200,799.0,0.2,3196.0,0.8,3995.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0
2,124C1F67-488B-4B55-B449-C45650BDECC7,4,7,1,536.0,536.0,0,0,0,1,1,1,1,0,1718.0,1150.0,20,1.0,0.0,0.0,21,0,1150.0,299.0,3.846,1150.0,1.0,0.0,0.0,1150.0,568.0,0,1150.0,1150.0,1.0,1150.0,1.000,1150.0,1.0,1150.0,1.0,1150.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0


In [64]:
parent_code=code
parent_code.columns=['parent_code', 'parent_code_total_sales_amount', 'parent_code_sales_7days', 'parent_code_ratio_7days',
       'parent_code_sales_14days', 'parent_code_ratio_14days', 'parent_code_sales_30days',
       'parent_code_ratio_30days', 'parent_code_sales_60days', 'parent_code_ratio_60days',
       'parent_code_sales_180days', 'parent_code_ratio_180days', 'parent_code_max_7days',
       'parent_code_min_7days', 'parent_code_median_7days', 'parent_code_mean_7days',
       'parent_code_max_14days', 'parent_code_min_14days', 'parent_code_median_14days',
       'parent_code_mean_14days', 'parent_code_max_30days', 'parent_code_min_30days',
       'parent_code_median_30days', 'parent_code_mean_30days', 'parent_code_max_60days',
       'parent_code_min_60days', 'parent_code_median_60days', 'parent_code_mean_60days',
       'parent_code_max_180days', 'parent_code_min_180days', 'parent_code_median_180days',
       'parent_code_mean_180days']
parent_code.head(3)

,parent_code,parent_code_total_sales_amount,parent_code_sales_7days,parent_code_ratio_7days,parent_code_sales_14days,parent_code_ratio_14days,parent_code_sales_30days,parent_code_ratio_30days,parent_code_sales_60days,parent_code_ratio_60days,parent_code_sales_180days,parent_code_ratio_180days,parent_code_max_7days,parent_code_min_7days,parent_code_median_7days,parent_code_mean_7days,parent_code_max_14days,parent_code_min_14days,parent_code_median_14days,parent_code_mean_14days,parent_code_max_30days,parent_code_min_30days,parent_code_median_30days,parent_code_mean_30days,parent_code_max_60days,parent_code_min_60days,parent_code_median_60days,parent_code_mean_60days,parent_code_max_180days,parent_code_min_180days,parent_code_median_180days,parent_code_mean_180days
0,0.0,18506893.0,5865675.0,0.317,8728702.0,0.472,12569867.0,0.679,15237826.0,0.823,17362861.0,0.938,9999.0,100.0,1028.524461,899.0,9999.0,100.0,1043.728566,899.0,16990.0,100.0,1103.200544,950.0,16990.0,100.0,1132.418698,989.0,16990.0,100.0,1140.118261,990.0
1,2.0,1515784.0,684725.0,0.452,970749.0,0.640,1187436.0,0.783,1247623.0,0.823,1382325.0,0.912,4038.0,149.0,541.712816,449.0,8076.0,149.0,555.030875,458.0,8076.0,99.0,560.375649,458.0,8076.0,99.0,561.739307,458.0,8076.0,99.0,569.326606,458.0
2,4.0,44808.0,1851.0,0.041,2631.0,0.059,4509.0,0.101,14144.0,0.316,32940.0,0.735,858.0,490.0,617.000000,503.0,858.0,390.0,526.200000,490.0,939.0,390.0,644.142857,503.0,1878.0,390.0,884.000000,898.5,1878.0,230.0,968.823529,939.0


In [65]:
df=pd.merge(df,parent_code)
df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,code,parent_code,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days,code_total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days,code_max_7days,code_min_7days,code_median_7days,code_mean_7days,code_max_14days,code_min_14days,code_median_14days,code_mean_14days,code_max_30days,code_min_30days,code_median_30days,code_mean_30days,code_max_60days,code_min_60days,code_median_60days,code_mean_60days,code_max_180days,code_min_180days,code_median_180days,code_mean_180days,parent_code_total_sales_amount,parent_code_sales_7days,parent_code_ratio_7days,parent_code_sales_14days,parent_code_ratio_14days,parent_code_sales_30days,parent_code_ratio_30days,parent_code_sales_60days,parent_code_ratio_60days,parent_code_sales_180days,parent_code_ratio_180days,parent_code_max_7days,parent_code_min_7days,parent_code_median_7days,parent_code_mean_7days,parent_code_max_14days,parent_code_min_14days,parent_code_median_14days,parent_code_mean_14days,parent_code_max_30days,parent_code_min_30days,parent_code_median_30days,parent_code_mean_30days,parent_code_max_60days,parent_code_min_60days,parent_code_median_60days,parent_code_mean_60days,parent_code_max_180days,parent_code_min_180days,parent_code_median_180days,parent_code_mean_180days
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,536.0,536.0,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.2,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0
1,9584B6E4-264C-4FFC-8EF6-5223301E692A,9,7,4,536.0,536.0,0,1,0,1,1,0,0,0,1380.0,799.0,20,1.0,0.0,0.0,81,0,799.0,299.0,2.672,3995.0,5.0,0.0,0.0,799.0,581.0,0,3995.0,799.0,0.2,799.0,0.200,799.0,0.2,3196.0,0.8,3995.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0
2,124C1F67-488B-4B55-B449-C45650BDECC7,4,7,1,536.0,536.0,0,0,0,1,1,1,1,0,1718.0,1150.0,20,1.0,0.0,0.0,21,0,1150.0,299.0,3.846,1150.0,1.0,0.0,0.0,1150.0,568.0,0,1150.0,1150.0,1.0,1150.0,1.000,1150.0,1.0,1150.0,1.0,1150.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946

## 觀察相關係數

In [66]:
corrtable=(df.corr()[df.corr().index=='sales_7days']).T
sales_7days=corrtable.sort_values(ascending=False,by='sales_7days')
sales_7days=sales_7days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_7days

,sales_7days
price_gap,0.092855
unit_price,0.087693
avg_unit_price,0.087487
item_price,0.081142
item_orig_price,0.078186
compare_cate_unit_price,0.052588
code_max_60days,0.049080
code_max_30days,0.045493
deny_installment,0.039579
code_median_7days,0.039174


In [67]:
corrtable=(df.corr()[df.corr().index=='sales_14days']).T
sales_14days=corrtable.sort_values(ascending=False,by='sales_14days')
sales_14days=sales_14days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_14days

,sales_14days
price_gap,0.100835
avg_unit_price,0.095470
unit_price,0.095354
item_price,0.091266
item_orig_price,0.087832
compare_cate_unit_price,0.057756
code_max_60days,0.053803
code_median_14days,0.049861
code_max_30days,0.049501
code_median_7days,0.049326


In [68]:
corrtable=(df.corr()[df.corr().index=='sales_30days']).T
sales_30days=corrtable.sort_values(ascending=False,by='sales_30days')
sales_30days=sales_30days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_30days

,sales_30days
price_gap,0.106340
avg_unit_price,0.105410
unit_price,0.105327
item_price,0.103596
item_orig_price,0.096455
compare_cate_unit_price,0.065042
code_median_14days,0.060582
code_median_30days,0.059866
code_median_7days,0.059829
code_median_60days,0.059735


In [69]:
corrtable=(df.corr()[df.corr().index=='sales_60days']).T
sales_60days=corrtable.sort_values(ascending=False,by='sales_60days')
sales_60days=sales_60days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_60days

,sales_60days
unit_price,0.114531
avg_unit_price,0.114444
item_price,0.113826
price_gap,0.108478
item_orig_price,0.103314
code_median_14days,0.072308
code_median_30days,0.071681
code_median_60days,0.071558
code_median_7days,0.071362
code_median_180days,0.070914


In [70]:
corrtable=(df.corr()[df.corr().index=='sales_180days']).T
sales_180days=corrtable.sort_values(ascending=False,by='sales_180days')
sales_180days=sales_180days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_180days

,sales_180days
item_price,0.117506
avg_unit_price,0.117474
unit_price,0.117419
price_gap,0.107218
item_orig_price,0.106314
code_median_14days,0.084806
code_median_30days,0.083953
code_median_60days,0.083722
code_median_7days,0.083392
code_median_180days,0.083207


## 因為code的類別數較大,故使用計數編碼

In [71]:
parent_code=df.parent_code.value_counts().to_frame()
parent_code=parent_code.rename(columns={'parent_code':'parent_code_counts'})
parent_code.index.name='parent_code'
#code.columns=[['code','code_counts']]
parent_code=parent_code.reset_index(drop=False)
df=pd.merge(df,parent_code,how='left')
df=df.drop(columns=['parent_code'])

df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,code,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days,code_total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days,code_max_7days,code_min_7days,code_median_7days,code_mean_7days,code_max_14days,code_min_14days,code_median_14days,code_mean_14days,code_max_30days,code_min_30days,code_median_30days,code_mean_30days,code_max_60days,code_min_60days,code_median_60days,code_mean_60days,code_max_180days,code_min_180days,code_median_180days,code_mean_180days,parent_code_total_sales_amount,parent_code_sales_7days,parent_code_ratio_7days,parent_code_sales_14days,parent_code_ratio_14days,parent_code_sales_30days,parent_code_ratio_30days,parent_code_sales_60days,parent_code_ratio_60days,parent_code_sales_180days,parent_code_ratio_180days,parent_code_max_7days,parent_code_min_7days,parent_code_median_7days,parent_code_mean_7days,parent_code_max_14days,parent_code_min_14days,parent_code_median_14days,parent_code_mean_14days,parent_code_max_30days,parent_code_min_30days,parent_code_median_30days,parent_code_mean_30days,parent_code_max_60days,parent_code_min_60days,parent_code_median_60days,parent_code_mean_60days,parent_code_max_180days,parent_code_min_180days,parent_code_median_180days,parent_code_mean_180days,parent_code_counts
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,536.0,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.2,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,52043
1,9584B6E4-264C-4FFC-8EF6-5223301E692A,9,7,4,536.0,0,1,0,1,1,0,0,0,1380.0,799.0,20,1.0,0.0,0.0,81,0,799.0,299.0,2.672,3995.0,5.0,0.0,0.0,799.0,581.0,0,3995.0,799.0,0.2,799.0,0.200,799.0,0.2,3196.0,0.8,3995.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,52043
2,124C1F67-488B-4B55-B449-C45650BDECC7,4,7,1,536.0,0,0,0,1,1,1,1,0,1718.0,1150.0,20,1.0,0.0,0.0,21,0,1150.0,299.0,3.846,1150.0,1.0,0.0,0.0,1150.0,568.0,0,1150.0,1150.0,1.0,1150.0,1.000,1150.0,1.0,1150.0,1.0,1150.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.94

In [72]:
code=df.code.value_counts().to_frame()
code=code.rename(columns={'code':'code_counts'})
code.index.name='code'
#code.columns=[['code','code_counts']]
code=code.reset_index(drop=False)
df=pd.merge(df,code,how='left')

df=df.drop(columns=['code'])
df.head()

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days,code_total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days,code_max_7days,code_min_7days,code_median_7days,code_mean_7days,code_max_14days,code_min_14days,code_median_14days,code_mean_14days,code_max_30days,code_min_30days,code_median_30days,code_mean_30days,code_max_60days,code_min_60days,code_median_60days,code_mean_60days,code_max_180days,code_min_180days,code_median_180days,code_mean_180days,parent_code_total_sales_amount,parent_code_sales_7days,parent_code_ratio_7days,parent_code_sales_14days,parent_code_ratio_14days,parent_code_sales_30days,parent_code_ratio_30days,parent_code_sales_60days,parent_code_ratio_60days,parent_code_sales_180days,parent_code_ratio_180days,parent_code_max_7days,parent_code_min_7days,parent_code_median_7days,parent_code_mean_7days,parent_code_max_14days,parent_code_min_14days,parent_code_median_14days,parent_code_mean_14days,parent_code_max_30days,parent_code_min_30days,parent_code_median_30days,parent_code_mean_30days,parent_code_max_60days,parent_code_min_60days,parent_code_median_60days,parent_code_mean_60days,parent_code_max_180days,parent_code_min_180days,parent_code_median_180days,parent_code_mean_180days,parent_code_counts,code_counts
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.2,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,52043,28492
1,9584B6E4-264C-4FFC-8EF6-5223301E692A,9,7,4,0,1,0,1,1,0,0,0,1380.0,799.0,20,1.0,0.0,0.0,81,0,799.0,299.0,2.672,3995.0,5.0,0.0,0.0,799.0,581.0,0,3995.0,799.0,0.2,799.0,0.200,799.0,0.2,3196.0,0.8,3995.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,52043,28492
2,124C1F67-488B-4B55-B449-C45650BDECC7,4,7,1,0,0,0,1,1,1,1,0,1718.0,1150.0,20,1.0,0.0,0.0,21,0,1150.0,299.0,3.846,1150.0,1.0,0.0,0.0,1150.0,568.0,0,1150.0,1150.0,1.0,1150.0,1.000,1150.0,1.0,1150.0,1.0,1150.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.9

In [73]:
df['orig_unit_price']=df['item_orig_price']/df['quantity_multiplier']

## 預測銷量

In [96]:
continousIndex=['item_orig_price','item_price','item_default_daily_amount',
                'subtotal_allguid','quantity_sold_allguid',
                #'ratio_7days','ratio_14days','ratio_30days','ratio_60days','ratio_180days'
                #'code_ratio_7days','code_ratio_14days','code_ratio_30days','code_ratio_60days','code_ratio_180days',
                'code_total_sales_amount',
                'code_sales_7days','code_sales_14days','code_sales_30days','code_sales_60days','code_sales_180days',
                'code_max_7days','code_min_7days','code_median_7days','code_mean_7days','code_max_14days','code_min_14days',
                'code_median_14days','code_mean_14days','code_max_30days','code_min_30days','code_median_30days',
                'code_mean_30days','code_max_60days','code_min_60days','code_median_60days','code_mean_60days',
                'code_max_180days','code_min_180days','code_median_180days','code_mean_180days','parent_code_total_sales_amount',
                #'parent_code_ratio_7days','parent_code_ratio_14days','parent_code_ratio_30days','parent_code_ratio_60days','parent_code_ratio_180days',
                'parent_code_sales_7days','parent_code_sales_14days','parent_code_sales_30days','parent_code_sales_60days',
                'parent_code_sales_180days','parent_code_max_7days','parent_code_min_7days',
                'parent_code_median_7days','parent_code_mean_7days','parent_code_max_14days','parent_code_min_14days',
                'parent_code_median_14days','parent_code_mean_14days','parent_code_max_30days','parent_code_min_30days',
                'parent_code_median_30days','parent_code_mean_30days','parent_code_max_60days','parent_code_min_60days',
                'parent_code_median_60days','parent_code_mean_60days','parent_code_max_180days','parent_code_min_180days',
                'parent_code_median_180days','parent_code_mean_180days','parent_code_counts','code_counts','orig_unit_price']

targetIndex=['guid_subtotal','sales_7days','sales_14days','sales_30days', 'sales_60days','sales_180days']
categoryIndex=['product_launch_month','product_launch_hour','product_launch_weekday','is_Presales','is_Longtern',
'enable_isp','is_Discount','installment_3months','installment_6months','installment_12months','deny_installment']

In [178]:
##將銷量相關取log
df_new=df.drop(columns='business_hour_guid')
ohe_df=pd.get_dummies(df_new,columns=categoryIndex)
ohe_df[continousIndex] = np.log1p(ohe_df[continousIndex])
ohe_df[targetIndex] = np.log1p(ohe_df[targetIndex])

In [248]:
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
def trainTestValSplit(df):
    # 將資料切分為訓練(80%),測試(10%)和驗證(10%)三份。
    train=df.sample(frac=0.8,random_state=0)
    test=df.drop(train.index )
    val=test.sample(frac=0.5,random_state=0)
    test=test.drop(val.index)
    return train,test,val

def dfXYSplit(df,targetIndex,targetName):
    # 將特徵和目標變數切成兩份資料。
    
    dfX=df.drop(columns=targetIndex)
    dfY=df[targetName]
    
    return dfX,dfY
def trainmodel(df,targetIndex,targetname,model):
    train,test,val=trainTestValSplit(df)
    trainX,trainY=dfXYSplit(train,targetIndex,targetname)
    testX,testY=dfXYSplit(test,targetIndex,targetname)
    valX,valY=dfXYSplit(val,targetIndex,targetname)
    model.fit(trainX,trainY)
    testPredY=model.predict(testX)
    valPredY=model.predict(valX)
    print(model)
    print("val_mean squared error: %.2f"
      % mean_squared_error(valY, valPredY))

    print("val_mean_absolute_error: %.2f"
      % mean_absolute_error(valY, valPredY))
    print("val_r2_score: %.2f"
      % r2_score(valY,valPredY))
    subtotal=pd.DataFrame()
    subtotal['predict']=np.expm1(np.array(testPredY.tolist()).reshape(-1))
    subtotal['real']=np.expm1(testY[testY.columns[0]].reset_index(drop=True))
    def check(x):
        if x <1 :
            return 0
        else:
            return x
    subtotal.predict=subtotal.predict.apply(check)

    print("test_mean squared error: %.2f"
      % mean_squared_error(subtotal['real'],subtotal['predict']))

    print("test_mean_absolute_error: %.2f"
      % mean_absolute_error(subtotal['real'],subtotal['predict']))
    print("test_r2_score: %.2f"
      % r2_score(subtotal['real'],subtotal['predict']))
    return subtotal

## DecisionTree

In [249]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=0)
targetname=['sales_7days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')
val_mean squared error: 0.00
val_mean_absolute_error: 0.01
val_r2_score: 1.00
test_mean squared error: 1941433.53
test_mean_absolute_error: 80.04
test_r2_score: 0.97


,predict,real
0,1199.0,1199.0
1,1298.0,1302.0
2,0.0,0.0
3,0.0,0.0
4,1797.0,1797.0
5,10850.0,10746.0


In [251]:
model = DecisionTreeRegressor(random_state=0)
targetname=['sales_14days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')
val_mean squared error: 0.00
val_mean_absolute_error: 0.01
val_r2_score: 1.00
test_mean squared error: 19436091.10
test_mean_absolute_error: 134.19
test_r2_score: 0.90


,predict,real
0,1199.0,1199.0
1,3897.0,3906.0
2,420.0,420.0
3,0.0,0.0
4,2396.0,2396.0
5,18522.0,17910.0


In [252]:
model = DecisionTreeRegressor(random_state=0)
targetname=['sales_30days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')
val_mean squared error: 0.00
val_mean_absolute_error: 0.00
val_r2_score: 1.00
test_mean squared error: 24940602.11
test_mean_absolute_error: 169.24
test_r2_score: 0.95


,predict,real
0,2396.655736,2398.0
1,6510.000000,6510.0
2,1260.009705,1260.0
3,0.000000,0.0
4,2994.243553,2995.0
5,27453.570170,27462.0


In [253]:
model = DecisionTreeRegressor(random_state=0)
targetname=['sales_60days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')
val_mean squared error: 0.00
val_mean_absolute_error: 0.00
val_r2_score: 1.00
test_mean squared error: 17610517.29
test_mean_absolute_error: 157.08
test_r2_score: 0.98


,predict,real
0,2398.329222,2398.0
1,6510.000000,6510.0
2,1259.921225,1260.0
3,899.000000,878.0
4,2994.245181,2995.0
5,27459.999781,27462.0


In [254]:
model = DecisionTreeRegressor(random_state=0)
targetname=['sales_180days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')
val_mean squared error: 0.00
val_mean_absolute_error: 0.00
val_r2_score: 1.00
test_mean squared error: 9644652.55
test_mean_absolute_error: 109.43
test_r2_score: 0.99


,predict,real
0,2398.321794,2398.0
1,6510.555468,6510.0
2,1259.909053,1260.0
3,4288.999883,4390.0
4,2994.778841,2995.0
5,27471.570160,27462.0


### XGBoost

In [211]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
targetname=['sales_7days']
xgb=trainmodel(ohe_df,targetIndex,targetname,model)
xgb.head(6)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
val_mean squared error: 0.01
val_mean_absolute_error: 0.04
val_r2_score: 1.00
test_mean squared error: 8868449.63
test_mean_absolute_error: 194.04
test_r2_score: 0.93


,predict,real
0,494.992167,449.0
1,0.000000,0.0
2,0.000000,0.0
3,4615.099902,5196.0
4,1646.930730,1500.0
5,0.000000,0.0


In [212]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
targetname=['sales_14days']
xgb=trainmodel(ohe_df,targetIndex,targetname,model)
xgb.head(6)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
val_mean squared error: 0.00
val_mean_absolute_error: 0.04
val_r2_score: 1.00
test_mean squared error: 18059198.55
test_mean_absolute_error: 310.52
test_r2_score: 0.95


,predict,real
0,0.000000,0.0
1,0.000000,0.0
2,0.000000,0.0
3,3425.555598,3408.0
4,249.900000,250.0
5,553.851126,537.0


In [213]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
targetname=['sales_30days']
xgb=trainmodel(ohe_df,targetIndex,targetname,model)
xgb.head(6)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
val_mean squared error: 0.00
val_mean_absolute_error: 0.03
val_r2_score: 1.00
test_mean squared error: 33522000.34
test_mean_absolute_error: 350.15
test_r2_score: 0.97


,predict,real
0,1009.376129,1043.0
1,3054.851793,3152.0
2,501.427778,490.0
3,3031.293628,2994.0
4,8637.398390,8670.0
5,14196.681150,15181.0


In [214]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
targetname=['sales_60days']
xgb=trainmodel(ohe_df,targetIndex,targetname,model)
xgb.head(6)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
val_mean squared error: 0.00
val_mean_absolute_error: 0.02
val_r2_score: 1.00
test_mean squared error: 101946159.41
test_mean_absolute_error: 384.31
test_r2_score: 0.94


,predict,real
0,1167.800281,1150.0
1,595.165426,596.0
2,1893.517658,1880.0
3,2219.887679,2394.0
4,0.000000,0.0
5,2652.704361,2748.0


In [215]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
targetname=['sales_180days']
xgb=trainmodel(ohe_df,targetIndex,targetname,model)
xgb.head(6)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
val_mean squared error: 0.00
val_mean_absolute_error: 0.02
val_r2_score: 1.00
test_mean squared error: 33614690.40
test_mean_absolute_error: 318.67
test_r2_score: 0.99


,predict,real
0,495.539037,498.0
1,0.000000,0.0
2,200.419037,199.0
3,239.278266,245.0
4,3236.496973,3240.0
5,1175.714251,1192.0


## RandomForest

In [216]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(max_depth=2, random_state=0,n_estimators=100)
targetname=['sales_7days']
rf=trainmodel(ohe_df,targetIndex,targetname,model)
rf.head(6)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)
val_mean squared error: 0.48
val_mean_absolute_error: 0.44
val_r2_score: 0.96
test_mean squared error: 146949792.97
test_mean_absolute_error: 1954.72
test_r2_score: 0.05


,predict,real
0,916.144735,1298.0
1,3065.220022,1098.0
2,0.000000,0.0
3,1035.412975,3195.0
4,0.000000,0.0
5,4317.177573,592752.0


## LinearRegression

In [217]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
targetname=['sales_7days']
lr=trainmodel(ohe_df,targetIndex,targetname,model)
lr.head(6)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
val_mean squared error: 5.11
val_mean_absolute_error: 1.94
val_r2_score: 0.62
test_mean squared error: 151504274943033536.00
test_mean_absolute_error: 3034532.50
test_r2_score: -1271478634.72


,predict,real
0,24.802337,1098.0
1,4.369510,0.0
2,0.000000,0.0
3,1067.895627,10080.0
4,4.179088,0.0
5,1.271104,0.0


## Ridge 

In [218]:
from sklearn.linear_model import Lasso, Ridge
model=Ridge()
targetname=['sales_7days']
rl=trainmodel(ohe_df,targetIndex,targetname,model)
rl.head(6)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
val_mean squared error: 5.03
val_mean_absolute_error: 1.93
val_r2_score: 0.62
test_mean squared error: 22048781929090.02
test_mean_absolute_error: 41907.71
test_r2_score: -187456.65


,predict,real
0,162.723505,2457.0
1,2467.404631,3195.0
2,1.301394,0.0
3,372.523946,1112.0
4,0.000000,0.0
5,1132.480334,1797.0


## 將資料切分訓練(80%),測試(10%)和驗證(10%)三份


* 未作細調參數前的MAE R2整理：

Model                    | Val_MAE                  |Val_R2                  | 
----------------         |:---------:               |---------:              | 
DecisionTreeRegressor    | 0.01/0.01/0.00/0.00/0.00 |1.0/1.0/1.0/1.0/1.0     |    0 | 
XGBRegressor             | 0.04/0.04/0.03/0.02/0.02 |1.0/1.0/1.0/1.0/1.0     |    0 |  
RandomForestRegressor    | 0.44                     |0.96                    |    0 |  
LinearRegression         | 1.94                     |0.62                    |    0 |  
Ridge                    | 1.93                     |0.62                    |    0 |  


* 因為所有數值型變數都有經過對數轉換，故在檢查測試集還原log後的表現，選擇以DecisionTree 與XGBregressor 進行參數調教

## 使用網格搜尋法
* 將資料重新分為訓練集與測試集

In [223]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
targetname=['sales_7days']
trainX,trainY=dfXYSplit(ohe_df,targetIndex,targetname)
x_train, x_test, y_train, y_test = train_test_split(trainX,trainY, test_size=0.2, random_state=4)

In [230]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
model =  xgb.XGBRegressor( )


# 設定要訓練的超參數組合
n_estimators = [10,25,50, 100, 150]
max_depth = [1, 3, 5,7,10]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(model, param_grid,n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train,y_train)
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 11.1min
/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory 

KeyboardInterrupt: 

In [234]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
model =  DecisionTreeRegressor()

# 設定要訓練的超參數組合
min_samples_split=[2,4,6,8,10]
min_samples_leaf=[1, 3,5,7,9]
max_depth = [1,3, 5,10,15,20]
param_grid = dict(max_depth=max_depth,min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(model, param_grid,n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train,y_train)
print("Best combination: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 150 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  7.5min finished


Best combination: 0.999931 using {'max_depth': 20, 'min_samples_leaf': 3, 'min_samples_split': 8}


In [255]:
model = DecisionTreeRegressor(random_state=0,max_depth=20,min_samples_leaf=3,min_samples_split=8)
targetname=['sales_7days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=3,
           min_samples_split=8, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')
val_mean squared error: 0.00
val_mean_absolute_error: 0.01
val_r2_score: 1.00
test_mean squared error: 3785882.32
test_mean_absolute_error: 85.19
test_r2_score: 0.95


,predict,real
0,1199.000000,1199.0
1,1308.464593,1302.0
2,0.000000,0.0
3,0.000000,0.0
4,1797.000000,1797.0
5,10788.848679,10746.0


In [260]:
model = DecisionTreeRegressor(random_state=0,max_depth=20,min_samples_leaf=3,min_samples_split=8)
targetname=['sales_14days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=3,
           min_samples_split=8, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')
val_mean squared error: 0.00
val_mean_absolute_error: 0.01
val_r2_score: 1.00
test_mean squared error: 9608770.75
test_mean_absolute_error: 114.42
test_r2_score: 0.95


,predict,real
0,1199.000000,1199.0
1,3914.980848,3906.0
2,420.831591,420.0
3,0.000000,0.0
4,2396.000000,2396.0
5,18318.422564,17910.0


In [261]:
model = DecisionTreeRegressor(random_state=0,max_depth=20,min_samples_leaf=3,min_samples_split=8)
targetname=['sales_30days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=3,
           min_samples_split=8, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')
val_mean squared error: 0.00
val_mean_absolute_error: 0.00
val_r2_score: 1.00
test_mean squared error: 38738481.51
test_mean_absolute_error: 185.14
test_r2_score: 0.93


,predict,real
0,2396.655736,2398.0
1,6508.142189,6510.0
2,1260.009705,1260.0
3,0.000000,0.0
4,2994.243553,2995.0
5,27457.999534,27462.0


In [258]:
model = DecisionTreeRegressor(random_state=0,max_depth=20,min_samples_leaf=3,min_samples_split=8)
targetname=['sales_60days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=3,
           min_samples_split=8, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')
val_mean squared error: 0.00
val_mean_absolute_error: 0.00
val_r2_score: 1.00
test_mean squared error: 29491498.45
test_mean_absolute_error: 162.35
test_r2_score: 0.96


,predict,real
0,2398.329222,2398.0
1,6508.142189,6510.0
2,1260.008617,1260.0
3,894.435512,878.0
4,2994.245181,2995.0
5,27459.999781,27462.0


In [259]:
model = DecisionTreeRegressor(random_state=0,max_depth=20,min_samples_leaf=3,min_samples_split=8)
targetname=['sales_180days']
dt=trainmodel(ohe_df,targetIndex,targetname,model)
dt.head(6)

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=3,
           min_samples_split=8, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')
val_mean squared error: 0.00
val_mean_absolute_error: 0.00
val_r2_score: 1.00
test_mean squared error: 7996863.11
test_mean_absolute_error: 94.60
test_r2_score: 0.99


,predict,real
0,2398.321794,2398.0
1,6510.555468,6510.0
2,1259.909053,1260.0
3,4343.073811,4390.0
4,2994.410918,2995.0
5,27462.000000,27462.0
